# Estimating VaR in EURUSD from IV using ML and QR

## Modeling-Ensemble

### Data Preparation

In [1]:
import pandas as pd
import numpy as np
import pickle
import joblib
import statsmodels.api as sm

In [2]:
with open('data_scale.pickle', 'rb') as f:
    data_scale = pickle.load(f)

with open('model_qr.pickle', 'rb') as f:
    model_qr = pickle.load(f)

with open('model_qr2.pickle', 'rb') as f:
    model_qr2 = pickle.load(f)

### QR-IM generated dataset

In [3]:
quantiles = [0.01, 0.025, 0.05, 0.95, 0.975, 0.99]

In [4]:
df_spot_is_x = data_scale['df_spot_is'].iloc[:,:-1]
df_spot2_is_x = data_scale['df_spot2_is'].iloc[:,:-1]
df_spread_is_x = data_scale['df_spread_is'].iloc[:,:-1]

In [5]:
predict_qr = dict()
for quantile, model in model_qr.items():
    predict_qr[quantile] = model.predict(df_spot_is_x[['IV_ATM', 'RR_25D']])

predict_qr2 = dict()
for quantile, model in model_qr2.items():
    predict_qr2[quantile] = model.predict(df_spot2_is_x)

### Random Forest: QR-IM dataset

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error

In [7]:
rf_models = dict()
for quantile in quantiles:
    print(f'###### Random Forest-Quantile {quantile} ######')
    X = df_spread_is_x
    y = predict_qr[quantile]

    # more sample weight toward recent data
    sample_weights = np.arange(1,len(y)+1)

    param_grid = {
       'n_estimators': [50, 100, 200],
       'max_depth': [None, 10, 20],
       'min_samples_split': [2, 5, 10],
       'min_samples_leaf': [1, 2, 4]
    }

    tscv = TimeSeriesSplit(n_splits=5)

    best_params = []
    best_scores = []

    # walk forward cv
    split = 0
    for train_index, test_index in tscv.split(X):
        # train test split
        split += 1
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        weights_train = sample_weights[train_index]
        weights_test = sample_weights[test_index]

        # random forest regression
        rf = RandomForestRegressor()
        grid_search = GridSearchCV(rf,
                                   param_grid,
                                   cv=TimeSeriesSplit(n_splits=3),
                                   n_jobs=-1)
        grid_search.fit(X_train, y_train,
                        sample_weight=weights_train)

        # hyper-parameter tuning
        best_params.append(grid_search.best_params_)
        best_rf = grid_search.best_estimator_
        y_pred = best_rf.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        best_scores.append(mse)
        print(f'Split {split} MSE: {mse}')

    # final rf model
    mean_score = np.mean(best_scores)
    final_rf = RandomForestRegressor(**best_params[-1])
    final_rf.fit(X, y, sample_weight=sample_weights)
    y_pred = final_rf.predict(X)
    mse = mean_squared_error(y, y_pred)
    print(f'Quantile {quantile} Final MSE: {mse}')
    
    # append final model
    rf_models[quantile] = final_rf

###### Random Forest-Quantile 0.01 ######
Split 1 MSE: 1.5372573898666862e-06
Split 2 MSE: 1.3769145815939367e-06
Split 3 MSE: 3.3664631710369686e-07
Split 4 MSE: 1.0985739536903793e-06
Split 5 MSE: 1.5561282695046664e-06
Quantile 0.01 Final MSE: 2.222583260116153e-08
###### Random Forest-Quantile 0.025 ######
Split 1 MSE: 8.187781523877505e-07
Split 2 MSE: 6.659763101596655e-07
Split 3 MSE: 1.3845404744418917e-07
Split 4 MSE: 5.419814389889644e-07
Split 5 MSE: 7.430899463823402e-07
Quantile 0.025 Final MSE: 1.4767511204939047e-08
###### Random Forest-Quantile 0.05 ######
Split 1 MSE: 5.07264761112262e-07
Split 2 MSE: 2.5215753113659697e-07
Split 3 MSE: 8.474787638059223e-08
Split 4 MSE: 2.443515531098794e-07
Split 5 MSE: 3.097383569878273e-07
Quantile 0.05 Final MSE: 1.0386889337692175e-08
###### Random Forest-Quantile 0.95 ######
Split 1 MSE: 7.237117932229468e-07
Split 2 MSE: 3.508318796746841e-08
Split 3 MSE: 3.855484463910626e-08
Split 4 MSE: 3.81808961624331e-08
Split 5 MSE: 2.97

In [8]:
joblib.dump(rf_models, 'rf_models.pkl')

['rf_models.pkl']

### XGBoost(1): QR-IM Dataset

In [9]:
import xgboost as xgb

In [10]:
xgb1_models = dict()
for quantile in quantiles:
    print(f'###### XGBoost 1-Quantile {quantile} ######')
    X = df_spread_is_x
    y = predict_qr[quantile]

    # more sample weight toward recent data
    sample_weights = np.arange(1,len(y)+1)

    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 0.9, 1.0]
    }

    tscv = TimeSeriesSplit(n_splits=5)

    best_params = []
    best_scores = []

    # walk forward cv
    split = 0
    for train_index, test_index in tscv.split(X):
        # train test split
        split += 1
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        weights_train = sample_weights[train_index]
        weights_test = sample_weights[test_index]

        # xgboost regression
        xgb_reg = xgb.XGBRegressor()
        grid_search = GridSearchCV(xgb_reg,
                                   param_grid,
                                   cv=TimeSeriesSplit(n_splits=3),
                                   n_jobs=-1)
        grid_search.fit(X_train, y_train,
                        sample_weight=weights_train)

        # hyper-parameter tuning
        best_params.append(grid_search.best_params_)
        best_xgb_reg = grid_search.best_estimator_
        y_pred = best_xgb_reg.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        best_scores.append(mse)
        print(f'Split {split} MSE: {mse}')

    # final xgboost model
    mean_score = np.mean(best_scores)
    final_xgb_reg = xgb.XGBRegressor(**best_params[-1])
    final_xgb_reg.fit(X, y, sample_weight=sample_weights)
    y_pred = final_xgb_reg.predict(X)
    mse = mean_squared_error(y, y_pred)
    print(f'Quantile {quantile} Final MSE: {mse}')
    
    # append final model
    xgb1_models[quantile] = final_xgb_reg

###### XGBoost 1-Quantile 0.01 ######
Split 1 MSE: 1.4538978117731475e-06
Split 2 MSE: 4.75001941221467e-07
Split 3 MSE: 1.8595648193373418e-07
Split 4 MSE: 3.1902499504692565e-07
Split 5 MSE: 1.270345096580873e-06
Quantile 0.01 Final MSE: 3.73324165510311e-09
###### XGBoost 1-Quantile 0.025 ######
Split 1 MSE: 1.1013882985999929e-06
Split 2 MSE: 2.966276045765665e-07
Split 3 MSE: 1.1210354686473428e-07
Split 4 MSE: 1.8390904552084965e-07
Split 5 MSE: 6.648778214190647e-07
Quantile 0.025 Final MSE: 1.1314575891788625e-09
###### XGBoost 1-Quantile 0.05 ######
Split 1 MSE: 6.776148489626505e-07
Split 2 MSE: 1.1561446876514746e-07
Split 3 MSE: 5.5242373590808064e-08
Split 4 MSE: 5.5743852018208567e-08
Split 5 MSE: 2.3833745400932406e-07
Quantile 0.05 Final MSE: 1.15565290844337e-09
###### XGBoost 1-Quantile 0.95 ######
Split 1 MSE: 6.015262453932209e-07
Split 2 MSE: 2.353188681254225e-08
Split 3 MSE: 4.0422646843336135e-08
Split 4 MSE: 1.6837918919426172e-08
Split 5 MSE: 2.42577002107031e

In [11]:
joblib.dump(xgb1_models, 'xgb1_models.pkl')

['xgb1_models.pkl']

### XGBoost(2): Gradient Boosting generated Dataset

In [12]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import make_scorer

#### Gradient Boosting Dataset Generation

In [13]:
import warnings
warnings.filterwarnings(action='ignore')

In [14]:
sample_weights = np.arange(1,len(y)+1)

param_grid = {
   'n_estimators': [50, 100, 200],
   'max_depth': [3, 5, 7],
   'learning_rate': [0.01, 0.1, 0.2],
   'subsample': [0.8, 0.9, 1.0]
}

def pinball_loss(y_true, y_pred, quantile):
    errors = y_true - y_pred
    return np.maximum((quantile - 1) * errors, quantile * errors).mean()

pinball_scorer = make_scorer(pinball_loss, greater_is_better=False)

tscv = TimeSeriesSplit(n_splits=5)

best_params = {q: [] for q in quantiles}
best_scores = {q: [] for q in quantiles}

X = df_spread_is_x
y = data_scale['df_spread_is'].iloc[:,-1]

# walk forward cv
split = 0
for train_index, test_index in tscv.split(X):
    split += 1
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    weights_train = sample_weights[train_index]
    weights_test = sample_weights[test_index]

    for quantile in quantiles:
        # gradient boosting quantile regressor
        gb_quantile = GradientBoostingRegressor(loss='quantile',
                                                alpha=quantile)
        grid_search = GridSearchCV(gb_quantile, param_grid,
                                   cv=TimeSeriesSplit(n_splits=3),
                                   n_jobs=-1, scoring=pinball_scorer)
        grid_search.fit(X_train, y_train, sample_weight=weights_train)

        # hyper-parameter tuning
        best_params[quantile].append(grid_search.best_params_)
        best_gb_quantile = GradientBoostingRegressor(
            loss='quantile', alpha=quantile, **grid_search.best_params_)
        best_gb_quantile.fit(X_train, y_train)
        y_pred = best_gb_quantile.predict(X_test)
        pinball = pinball_loss(y_test, y_pred, quantile)
        best_scores[quantile].append(pinball)
        print(f'{quantile} Quantile Split {split} Pinball: {pinball}')

# append final gb model
mean_scores = {q: np.mean(scores) for q, scores in best_scores.items()}

final_models = dict()
for quantile in quantiles:
    final_gb_quantile = GradientBoostingRegressor(
        loss='quantile', alpha=quantile, **best_params[quantile][-1])
    final_gb_quantile.fit(X, y, sample_weight=sample_weights)
    print(f'{quantile} Quantile Final Pinball: {pinball}')
    final_models[quantile] = final_gb_quantile

# generate target dataset
predict_gb = dict()
for quantile, model in final_models.items():
    predict_gb[quantile] = model.predict(X)

0.01 Quantile Split 1 Pinball: 0.0002378448149821997
0.025 Quantile Split 1 Pinball: 0.00042888888875717006
0.05 Quantile Split 1 Pinball: 0.0006942403629171711
0.95 Quantile Split 1 Pinball: 0.000650690898486907
0.975 Quantile Split 1 Pinball: 0.00035494438381243837
0.99 Quantile Split 1 Pinball: 0.00016030157935206592
0.01 Quantile Split 2 Pinball: 0.00015327019836672786
0.025 Quantile Split 2 Pinball: 0.00031679740669540555
0.05 Quantile Split 2 Pinball: 0.0005492848940740808
0.95 Quantile Split 2 Pinball: 0.0005536378095091283
0.975 Quantile Split 2 Pinball: 0.00031995668921588183
0.99 Quantile Split 2 Pinball: 0.0001545547052830905
0.01 Quantile Split 3 Pinball: 0.0001947464338308918
0.025 Quantile Split 3 Pinball: 0.0003836716748994674
0.05 Quantile Split 3 Pinball: 0.0006162678308358285
0.95 Quantile Split 3 Pinball: 0.0006941101193075007
0.975 Quantile Split 3 Pinball: 0.0004049889469382913
0.99 Quantile Split 3 Pinball: 0.0001762919434705711
0.01 Quantile Split 4 Pinball: 0.00

#### XGBoost Regression

In [15]:
xgb2_models = dict()
for quantile in quantiles:
    print(f'###### XGBoost 2-Quantile {quantile} ######')
    X = df_spread_is_x
    y = pd.Series(predict_gb[quantile],
                  index=df_spread_is_x.index)

    # more sample weight toward recent data
    sample_weights = np.arange(1,len(y)+1)

    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 0.9, 1.0]
    }

    tscv = TimeSeriesSplit(n_splits=5)

    best_params = []
    best_scores = []

    # walk forward cv
    split = 0
    for train_index, test_index in tscv.split(X):
        # train test split
        split += 1
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        weights_train = sample_weights[train_index]
        weights_test = sample_weights[test_index]

        # xgboost regression
        xgb_reg = xgb.XGBRegressor()
        grid_search = GridSearchCV(xgb_reg,
                                   param_grid,
                                   cv=TimeSeriesSplit(n_splits=3),
                                   n_jobs=-1)
        grid_search.fit(X_train, y_train,
                        sample_weight=weights_train)

        # hyper-parameter tuning
        best_params.append(grid_search.best_params_)
        best_xgb_reg = grid_search.best_estimator_
        y_pred = best_xgb_reg.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        best_scores.append(mse)
        print(f'Split {split} MSE: {mse}')

    # final xgboost model
    mean_score = np.mean(best_scores)
    final_xgb_reg = xgb.XGBRegressor(**best_params[-1])
    final_xgb_reg.fit(X, y, sample_weight=sample_weights)
    y_pred = final_xgb_reg.predict(X)
    mse = mean_squared_error(y, y_pred)
    print(f'Quantile {quantile} Final MSE: {mse}')
    
    # append final model
    xgb2_models[quantile] = final_xgb_reg

###### XGBoost 2-Quantile 0.01 ######
Split 1 MSE: 1.2311076090509598e-06
Split 2 MSE: 1.4220845725933297e-09
Split 3 MSE: 2.133823595126439e-09
Split 4 MSE: 8.189131011101105e-08
Split 5 MSE: 3.6296829162357874e-07
Quantile 0.01 Final MSE: 4.1795049736772594e-10
###### XGBoost 2-Quantile 0.025 ######
Split 1 MSE: 1.8113438297072164e-06
Split 2 MSE: 4.253891751419835e-08
Split 3 MSE: 7.4087652114714755e-09
Split 4 MSE: 1.1007960470231408e-07
Split 5 MSE: 2.838393889900539e-08
Quantile 0.025 Final MSE: 1.8450699658662916e-08
###### XGBoost 2-Quantile 0.05 ######
Split 1 MSE: 1.0684456037104718e-07
Split 2 MSE: 4.707845043784861e-09
Split 3 MSE: 1.3322735505801273e-09
Split 4 MSE: 4.495082448522923e-08
Split 5 MSE: 1.8001521006539026e-08
Quantile 0.05 Final MSE: 7.252422511738843e-11
###### XGBoost 2-Quantile 0.95 ######
Split 1 MSE: 2.204245096612315e-07
Split 2 MSE: 4.066226356678968e-09
Split 3 MSE: 6.327930314627804e-08
Split 4 MSE: 3.423943381369469e-08
Split 5 MSE: 2.28321811283470

In [16]:
joblib.dump(xgb2_models, 'xgb2_models.pkl')

['xgb2_models.pkl']

### LightGBM(1): QR-IM Dataset

In [17]:
import lightgbm as lgb

In [18]:
lgb1_models = dict()
for quantile in quantiles:
    print(f'###### LightGBM 1-Quantile {quantile} ######')
    X = df_spread_is_x
    y = predict_qr[quantile]

    # more sample weight toward recent data
    sample_weights = np.arange(1,len(y)+1)

    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 0.9, 1.0],
        'verbose': [-1]
    }

    tscv = TimeSeriesSplit(n_splits=5)

    best_params = []
    best_scores = []

    # walk forward cv
    split = 0
    for train_index, test_index in tscv.split(X):
        # train test split
        split += 1
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        weights_train = sample_weights[train_index]
        weights_test = sample_weights[test_index]

        # light gbm regression
        lgb_reg = lgb.LGBMRegressor()
        grid_search = GridSearchCV(lgb_reg,
                                   param_grid,
                                   cv=TimeSeriesSplit(n_splits=3),
                                   n_jobs=-1)
        grid_search.fit(X_train, y_train,
                        sample_weight=weights_train)

        # hyper-parameter tuning
        best_params.append(grid_search.best_params_)
        best_lgb_reg = grid_search.best_estimator_
        y_pred = best_lgb_reg.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        best_scores.append(mse)
        print(f'Split {split} MSE: {mse}')

    # final light gbm model
    mean_score = np.mean(best_scores)
    final_lgb_reg = lgb.LGBMRegressor(**best_params[-1])
    final_lgb_reg.fit(X, y, sample_weight=sample_weights)
    y_pred = final_lgb_reg.predict(X)
    mse = mean_squared_error(y, y_pred)
    print(f'Quantile {quantile} Final MSE: {mse}')
    
    # append final model
    lgb1_models[quantile] = final_lgb_reg

###### LightGBM 1-Quantile 0.01 ######
Split 1 MSE: 1.790465551848179e-06
Split 2 MSE: 9.627052156662542e-07
Split 3 MSE: 3.597757162072677e-07
Split 4 MSE: 8.675631925287862e-07
Split 5 MSE: 1.296956982940743e-06
Quantile 0.01 Final MSE: 2.6685382721792005e-08
###### LightGBM 1-Quantile 0.025 ######
Split 1 MSE: 1.264882471472374e-06
Split 2 MSE: 5.098047351216236e-07
Split 3 MSE: 2.2073421165387137e-07
Split 4 MSE: 5.021876463981952e-07
Split 5 MSE: 5.843591095688133e-07
Quantile 0.025 Final MSE: 9.975297604435664e-09
###### LightGBM 1-Quantile 0.05 ######
Split 1 MSE: 8.716430045283189e-07
Split 2 MSE: 2.5328299198276267e-07
Split 3 MSE: 1.153239065355161e-07
Split 4 MSE: 1.8748413253295058e-07
Split 5 MSE: 2.2849973482750662e-07
Quantile 0.05 Final MSE: 5.287941503344655e-09
###### LightGBM 1-Quantile 0.95 ######
Split 1 MSE: 8.911824249584444e-07
Split 2 MSE: 1.17292202350402e-07
Split 3 MSE: 9.270400134322852e-08
Split 4 MSE: 3.003311297913809e-08
Split 5 MSE: 1.8205264568242003e

In [19]:
joblib.dump(lgb1_models, 'lgb1_models.pkl')

['lgb1_models.pkl']

### LightGBM(2): QR-IM Dataset with Interaction

In [20]:
lgb2_models = dict()
for quantile in quantiles:
    print(f'###### LightGBM 2-Quantile {quantile} ######')
    y = predict_qr2[quantile]
    X = df_spread_is_x[df_spread_is_x.index>=y.index[0]]

    # more sample weight toward recent data
    sample_weights = np.arange(1,len(y)+1)

    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 0.9, 1.0],
        'verbose': [-1]
    }

    tscv = TimeSeriesSplit(n_splits=5)

    best_params = []
    best_scores = []

    # walk forward cv
    split = 0
    for train_index, test_index in tscv.split(X):
        # train test split
        split += 1
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        weights_train = sample_weights[train_index]
        weights_test = sample_weights[test_index]

        # light gbm regression
        lgb_reg = lgb.LGBMRegressor()
        grid_search = GridSearchCV(lgb_reg,
                                   param_grid,
                                   cv=TimeSeriesSplit(n_splits=3),
                                   n_jobs=-1)
        grid_search.fit(X_train, y_train,
                        sample_weight=weights_train)

        # hyper-parameter tuning
        best_params.append(grid_search.best_params_)
        best_lgb_reg = grid_search.best_estimator_
        y_pred = best_lgb_reg.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        best_scores.append(mse)
        print(f'Split {split} MSE: {mse}')

    # final light gbm model
    mean_score = np.mean(best_scores)
    final_lgb_reg = lgb.LGBMRegressor(**best_params[-1])
    final_lgb_reg.fit(X, y, sample_weight=sample_weights)
    y_pred = final_lgb_reg.predict(X)
    mse = mean_squared_error(y, y_pred)
    print(f'Quantile {quantile} Final MSE: {mse}')
    
    # append final model
    lgb2_models[quantile] = final_lgb_reg

###### LightGBM 2-Quantile 0.01 ######
Split 1 MSE: 1.1085141491131679e-05
Split 2 MSE: 7.348411305157841e-06
Split 3 MSE: 1.0134850772903324e-05
Split 4 MSE: 1.653311672308595e-05
Split 5 MSE: 8.175006581755998e-06
Quantile 0.01 Final MSE: 5.029330781105214e-07
###### LightGBM 2-Quantile 0.025 ######
Split 1 MSE: 7.020032761227292e-06
Split 2 MSE: 7.726545378044094e-06
Split 3 MSE: 6.197562497218393e-06
Split 4 MSE: 2.286527481739401e-05
Split 5 MSE: 9.24679587853899e-06
Quantile 0.025 Final MSE: 1.6247147942826845e-06
###### LightGBM 2-Quantile 0.05 ######
Split 1 MSE: 3.278468857751274e-06
Split 2 MSE: 3.0390651476461033e-06
Split 3 MSE: 3.3969692112810778e-06
Split 4 MSE: 5.3624084201086635e-06
Split 5 MSE: 2.7638088528592243e-06
Quantile 0.05 Final MSE: 2.8768163274020064e-07
###### LightGBM 2-Quantile 0.95 ######
Split 1 MSE: 4.346665866513345e-06
Split 2 MSE: 2.3928797244780634e-06
Split 3 MSE: 1.0850504979028253e-05
Split 4 MSE: 8.283108506942155e-06
Split 5 MSE: 6.843265742394

In [21]:
joblib.dump(lgb2_models, 'lgb2_models.pkl')

['lgb2_models.pkl']